# Chinook Records Store Database

In this project, we will be using SQL to answer business questions. 
The database used is a modified version of chinook.db, which is a sample database available in several database systems. The Chinook data represents a digital media store containing information about artists, their albums, sales information on both albums & tracks as well as customers and employee data. A schema of the database and its eleven tables is provided below.

![Chinook](chinook.svg)

Using the chinook database, we will answer questions such as which country recorded the highest sales, what are the popular genres of music, etc. Purposely, only SQL is used for computation in this notebook as a showcase. Pandas and matplotlib are used for displaying data.


In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [6]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Let's first check the tables provided within our database:

In [7]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Which of the 4 albums should Chinook add to the store?

The Chinook record store just signed a deal with a new record label that focuses on artists in the USA. We have been tasked to select 3 of the 4 albums listed below. 

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Metero and the Girls | Pop |
| Slim Jim Bites | Blues |


We have the artists' names and genre of music they produce. To answer the question above, we should look at the most popular genres in the USA. To do this, we'll find the total tracks sold by each genre in the USA. We can retrieve this information from the `customer`, `invoice`, `invoice_line`, `track`, and `genre` tables by using multiple `LEFT JOIN` keywords.

In [25]:
%%sql
   SELECT g.name AS genre,
          COUNT(*) AS total
     FROM invoice_line AS il
LEFT JOIN track AS t ON il.track_id = t.track_id
LEFT JOIN genre AS g ON g.genre_id = t.genre_id
LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = "USA" 
 GROUP BY g.name
 ORDER BY total DESC
    LIMIT 10;      

 * sqlite:///chinook.db
Done.


genre,total
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Pop,22
Latin,22
Hip Hop/Rap,20
Jazz,14


Given our findings, we can update our last table with the genre rankings:

| Artist Name | Genre | Genre Rank |
| --- | --- | --- |
| Regal | Hip-Hop | 9 |
| Red Tone | Punk | 2 |
| Metero and the Girls | Pop | 8 |
| Slim Jim Bites | Blues | 5 |

We should consequently advise Chinook to add **Red Tone's, Metero and the Girls', and Slim Jim Bites' albums**. However, we can see that **Rock** is by far the most sold genre in the USA, so one recommendation would be to have Chinook look into having more rock artists and albums. 

## Exploring Customer Purchases Based on Employees

Whenever a customer first makes a purchase, they are assigned to a sales support representative. We were tasked to analyze employee performance based on the number of and total sales each one has. We can retrieve this information through the `employee`, `customer`, `invoice`, and `invoice_line` tables.

In [40]:
%%sql
   SELECT e.first_name || " " || e.last_name AS name,
          ROUND(SUM(i.total), 2) AS total_sales,
          COUNT(*) AS nb_sales,
          STRFTIME('%d/%m/%Y', e.hire_date) AS hiring_date
     FROM invoice AS i
LEFT JOIN customer AS c ON c.customer_id = i.customer_id
LEFT JOIN employee AS e ON e.employee_id = c.support_rep_id
 GROUP BY name
 ORDER BY total DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


name,total_sales,nb_sales,hiring_date
Jane Peacock,1731.51,212,01/04/2017
Steve Johnson,1393.92,188,17/10/2017
Margaret Park,1584.0,214,03/05/2017


We can see that all sales agents were hired in 2017. **Jane Peacock** had the most number of transactions and total sales. Not surprisingly, the earlier the hire date, the more transactions and total sales the sales agent had.

## Analyzing Customers From Different Countries

Next, we can analyze the sales data for customers from each country. Specifically, we can look at the total number of customers, total value of sales, average value of sales per customer, and average order value. We can retrieve these values from the `customers` and `invoice` tables. 

**Important note**: 
- We are going to use the `country` value in the `customers` table rather than the `invoice` table because we are analyzing ALL customers rather than the customers who made a purchase.
- A country that has exactly 1 customer is going to be classified as `Other` in order not to .

In [41]:
%%sql
WITH country_or_other_facts AS
     (
     SELECT CASE
            WHEN (
                  SELECT count(*)
                    FROM customer
                   WHERE country = c.country
                 ) = 1 THEN "Other"
            ELSE c.country
            END AS country,
            c.customer_id,
            i.invoice_id,
            i.total
       FROM invoice AS i
            LEFT JOIN customer AS c 
            ON c.customer_id = i.customer_id
     )

SELECT country,
       customers,
       ROUND(total_sales,2) AS total_sales,
       ROUND(total_sales/customers,2) AS average_customer_value,
       ROUND(total_sales/invoices,2) AS average_order_value
FROM (  
        SELECT country,
               COUNT(DISTINCT customer_id) AS customers,
               COUNT(DISTINCT invoice_id) AS invoices,
               SUM(total) AS total_sales,
               CASE
               WHEN country = "Other" THEN 1
               ELSE 0
               END AS sort
          FROM country_or_other_facts
      GROUP BY country
     )
ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_customer_value,average_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


Based on our findings above, **USA** had the most customers and total sales. We can see a **positive correlation** between number of `customers` and `total sales`. Something interesting to point out is that Czech Republic's customers had the highest average for sales per customer and order value; however there were only 2 unique customers. Portugal had the 2nd highest average sales per customer, but the lowest average order value.

From this, we can recommend Chinook add the most popular genres in each country to diversify their target market or to find ways to continuously increase their sales in the USA. 

## Exploring albums vs individual track purchase

The Chinook store has 2 purchasing methods for the customers. They can either buy the whole album or purchase each individual track. Previously, Chinook did not allow customers to combine these types of purchases together. They are considering changing their purchasing strategy to save money by buying the most popular tracks from the albums rather than all of it.  

Some 'edge cases' we have to consider are that albums with 1 or 2 tracks are likely to be sold as individual tracks and customers may decide to purchase all tracks separately from an album and add a few individual tracks. We can ignore these 2 cases because we are focused on maximizing revenue and the company claims the 2nd case does not happen often.

We can answer the question above by finding the percentages of purchases (individual vs. album) and number of purchases for each. To figure out whether not the customer bought a whole album or individual tracks, we must compare two sets: the tracks from the album and the tracks purchased on one invoice.

### Set comparison in SQL
In order to compare two tables we have the following tools with SQL :
* UNION
* INTERSECT
* EXCEPT  
They only give information on the shared (or not) items in the tables though.

In order to assess equality between Table A and Table B there are 3 methods:
* 1: (A EXCEPT B IS NULL) AND (B EXCEPT A IS NULL)
* 2: (COUNT(A INTERSECT B) = COUNT(A)) AND (COUNT(A INTERSECT B) = COUNT(B))
* 3: A FULL JOIN B WHERE (A.Item IS NULL OR B.Item IS NULL)

Let's implement the first method on our database:

In [44]:
%%sql

WITH 

facts AS
(SELECT *
   FROM invoice_line AS il
        LEFT JOIN track AS t
        ON t.track_id = il.track_id
  GROUP BY invoice_id),

album_facts AS
(SELECT invoice_id,
        CASE
        WHEN (SELECT track_id
                FROM invoice_line
               WHERE invoice_id = facts.invoice_id
                     EXCEPT  
              SELECT track_id
                FROM track
               WHERE album_id = facts.album_id) IS NULL
             AND
              (SELECT track_id
                 FROM track
                WHERE album_id = facts.album_id
                     EXCEPT  
              SELECT track_id
                FROM invoice_line
               WHERE invoice_id = facts.invoice_id) IS NULL
        THEN "yes"
        ELSE "no"
        END AS album_purchase
FROM facts)

SELECT album_purchase,
       COUNT(*) AS nb_of_invoices,
       ROUND(CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*) FROM facts),2) 
       AS percentage_of_invoices
  FROM album_facts
 GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,nb_of_invoices,percentage_of_invoices
no,500,0.81
yes,114,0.19


Based on our findings above, customers buy **individual tracks** more frequently than **albums**, with each purchase type holding **81%** and **19%** of sales respectively. 

From this, we would recommend Chinook not to continue buying full albums from record companies, but rather buy the most popular tracks from each album. We could look at the top selling albums/tracks from each genre and advise Chinook to buy more of these specific tracks. 